# Item 25: Consider support for a non-throwing swap

## Discussion

- C++ STD Template Library std::swap is very powerful but the default implementation is inneficient for certain purposes.

    namespace std{
       template<typename T>
       void swap(T& a, T& b)
       {
          T temp(a);
          a = b;
          b = temp;
       }
    }

- For specific types, the standard std::swap will be extremely slow. By looking at the code above, you can see that it involves copying three objects.
- An example given is a type that consists of the design pattern called Pointer to Implementation[1] design approach.

[1]: http://en.cppreference.com/w/cpp/language/pimpl "Pointer to Implementation"

[Difference between Overloading and Specialization][1]
- Regular C++ clases do not overload. Class templates do not overload either.
- C++ funcitons having the same name do overload. Function templates are allowed to verload.

[1]:http://www.gotw.ca/publications/mill17.htm "Why not Specialize Funtion Templates"

### Code

---

## Example

In [7]:
#include <vector>
#include <iostream>
#include <algorithm>
#include <string>
#include <cstdlib>
#include <cxxabi.h>

In [8]:
namespace WidgetStuff {
    template<typename T>
    class WidgetImpl {
    private:
        std::vector<T> v;
    };

    template<typename T>
    class Widget {
    public:
        std::string type_name()
        {
            int status;
            std::string tname = typeid(T).name();
            char *demangled_name = abi::__cxa_demangle(tname.c_str(), NULL, NULL, &status);
            if(status == 0) {
                tname = demangled_name;
                std::free(demangled_name);
            }   
            return tname;
        }
    
        Widget &operator=(const Widget &rhs) {
            *pImpl = *(rhs.pImpl);
            return *this;
        }

        void swap(Widget &other) {
            std::cout << "Widget::swap\n";
            using std::swap;
            swap(pImpl, other.pImpl);
        }

    private:
        WidgetImpl<T> *pImpl;
    };

    template<typename T>
    void swap(Widget<T> &a, Widget<T> &b) {
        std::cout << "Widget::swap(Widget<T>& a, Widget<T>& b)\n";
        a.swap(b);
    }
      
}

In [9]:
template <typename T>
void doSomething(T& obj1, T& obj2)
{
  using std::swap;           // make std::swap available in this function
  swap(obj1, obj2);          // call the best swap for objects of type T
}

In [10]:
WidgetStuff::Widget<double> a;
WidgetStuff::Widget<double> b;

In [11]:
doSomething(a,b);

Widget::swap(Widget<T>& a, Widget<T>& b)
Widget::swap


(void) @0x7f9a39ff9c10


In [12]:
std::cout << a.type_name() << std::endl;
std::cout << b.type_name() << std::endl;

double
double


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f9a43813420


## Things to Remember

- Provide a swap member function when std:: swap would be inefficient for your type. Make sure your swap doesn't throw exceptions.
- If you offer a member swap, also offer a non-member swap that calls the member. For classes (not templates), sepcialize std::swap, too.
- When calling swap, employ a using declaration for std::swap, then call swap without nespace qualification..
- It's fine to totally specialize std templates for user-deined types, but never try to add something completely new to std.

## Sources 

1. [Why not Specialize Funtion Templates](http://www.gotw.ca/publications/mill17.htm) 
2. [Pointer to implementation - PImpl](http://en.cppreference.com/w/cpp/language/pimpl)